## Import and Initialize

In [ ]:
import brightway2 as bw
import functools
from bw2io import *
from bw2io.strategies import *

In [ ]:
# Enter the name of your project
bw.projects.set_current('lcis2024')

In [ ]:
# a user window will appear to select your import excel file. 

import select_file as select
database_path, database_name = select.file_selection()
# for checking the paths
print(database_path)
print(database_name)

### Create an ExcelImporter object

In [ ]:
ei = ExcelImporter(database_path)

### Apply the statistics and display statistics of matching

In [ ]:
ei.apply_strategies()
ei.statistics()

### Database matching
Perform a self matching and also match with other databases that are already present in the project. _note:_ a pop-up will appear to enter the number and name of the dataset dependencies manually. Please ensure to import these dependencies first before starting with the main dataset.

In [ ]:
ei.match_database(fields=["name"]) #database self matching

import db_dependencies as DB
for i in range (DB.db_array_length):
    ei.match_database(db_name=DB.db_names[i], fields=('name', 'reference product', 'location'))
#ei.match_database(None, ignore_categories=True)
ei.statistics()

### Printing the matched databases
**(only, if required)** To "neatly" print the entire imported data within ei. 


In [ ]:
import pprint
pprint.pprint(ei.data)

### Storing unlinked activities
**(only, if required)**
if there are any unlinked exchanges, the databases of these exchanges, and the unlinked exchanges are stored in their own arrays

In [ ]:
import pprint
unlinked_databases = []
unlinked_exchanges = []
for exc in ei.unlinked:
    if exc["database"] not in unlinked_databases:
        unlinked_databases.append(exc["database"])
    unlinked_exchanges.append(exc)
# the unlinked exchanges are stored as a list, where each element is a dict
print ("The unlinked database/s:\n",unlinked_databases)
print("The unlinked exchanges are:\n")
pprint.pprint(unlinked_exchanges)

### Exporting all matched activities to excel
**(only, if required)**
write an excel file with all the matched activities so far.copy the excel file that was written in the previous line to your desired location for further reference.


In [ ]:
ei.write_excel(only_unlinked=False)
ei.add_unlinked_activities()
ei.statistics()

### Dropping unlinked activities
**(Not recommended)** This is a last resort. It drops all the unlinked activities from the database.

In [ ]:
#uncomment the line below to enable the Nuclear option
ei.drop_unlinked(i_am_reckless=True)

### Final statistics check

In [ ]:
ei.statistics()

### Write database to brightway

In [ ]:
# will write to the database only if there are no unlinked exchanges
#writes the database into the bw2 as the same name of the excel sheet
ei.write_database()

**if unlinked (with ecoinvent) activities have been dropped**, then the code block below is used for searching and finding similar activities from the ecoinvent database.

**Note**: There is potentially one another method to do this. That is to search the unlinked_exchanges dict, and then add each activity by itself to the new database. However, this will result in loss of connection and attribution with ecoinvent database. May be, this method can be attempted in future. The below sections are still under construction. At present, I do not think there is a singular solution to import and address errors associatted with all sorts of dataset imports within brightway.

In [ ]:
print(unlinked_exchanges)
#print(unlinked_exchanges[0]["formula"])

In [ ]:
import bw2data as bd

#select the source ecoinvent database and the target database with missing activities
ecoinvent_db = bd.Database('cutoff391')
target_db = bd.Database('aircraftlca2')
#iterate through each unlinked exchanges stored in the unlinked_exchanges list
for exchange in unlinked_exchanges:
    #print (exchange["name"])

    #matches activity in unlinked_exchanges to those in th ecoinvent_db by name, location and reference product
    matches_activity = [act for act in ecoinvent_db if act["name"] == exchange["name"] and act["location"] == exchange["location"] and act["reference product"] == exchange["reference product"]]
    #matches_activity = bd.Database(database_name).search(exchange["name"])
    for match in matches_activity:
        #saves the activity code from the matched activity. This activity code is later used for copying the activity from ecoinvent db to target database
        activity_code = match["code"]
    #source_activity is the activity to be copies from ecoinvent
    source_activity = ecoinvent_db.get(activity_code)
    print("The activity from ecoinvent is \n", source_activity)

    #copies source activity to a variable called target_activity
    target_activity = source_activity.copy()
    target_activity["database"] = target_db.name

    # if there are any associatted formulas in the unlinked databases it will copied over to the newly added activities in the database. This copy may not work if the parameters are not available.
    if "formula" in exchange.keys():
        print("yes, there is formula")
        target_activity["formula"]=exchange["formula"]

    target_activity.save()
    target_db.process()

In [ ]:
quit()